<hr style="color:#a4342d;">
<p align="center">
    <b style="font-size:2.5vw; color:#a4342d; font-weight:bold;">
    Introduction to machine learning - Homework 3
    </b>
</p>
<hr style="color:#a4342d;">

<b>Authors</b>: <i>C. Bosch, M. Cornet & V. Mangeleer</i>

[comment]: <> (Section)
<hr style="color:#a4342d;">
<p align="center">
    <b style="font-size:1.5vw; color:#a4342d;">
    Initialization
    </b>
</p>
<hr style="color:#a4342d;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to initialize all the librairies needed and load the untouched dataset.